## Decode_re_tax_rolls_PDFs

E. Quinn 5/7/2021

This notebook uses pdfminer to extract the information from the EG real estate tax rolls

The documentation for pdfminer is at:

https://buildmedia.readthedocs.org/media/pdf/pdfminer-docs/latest/pdfminer-docs.pdf

Maintenance:


## Import standard python datascience packages

In [1]:
import sys
import math
import re
import copy
import numpy as np
import scipy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
%matplotlib inline

In [2]:
from datetime import datetime, timedelta, date
from datascience import *
import uuid
import random

### Show the directory we are running in

In [3]:
!pwd

/home/gquinn/EG/notebooks


## Import pdfminer packages

In [4]:
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice
from pdfminer.layout import LAParams
from pdfminer.converter import PDFPageAggregator
from pdfminer.pdfpage import PDFPage
from pdfminer.layout import LTTextBoxHorizontal

## Read the pdf and create a dictionary with the contents of each text box

### Function read_pdf() reads a PDF and returns a dictionary containing the contents

Strategy for this document:  

Save information from each element in the LTTextBox objects in a dictionary including:

- x0 horizontal coordinate of the upper left corner of the text box
- x1 horizontal coordinate of the lower right corner of the text box
- y0 vertical coordinate of the upper left corner of the text box
- y1 vertical coordinate of the lower right corner of the text box
- page number 
- sequence number of text box within this page
- text contained in the text box, converted to ascii

In [5]:
def read_pdf(path):
    document = open(path, 'rb')                                     #read a pdf and create a document object
    rsrcmgr = PDFResourceManager()                                  #create a resource manager
    laparams = LAParams()                                           #set the parameters for analysis
    device = PDFPageAggregator(rsrcmgr, laparams=laparams)          #create a PDF page aggregator object
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    
    pdf={}                                                          #dictionary to hold the results

    pageno = -1                                                     #initialize page coounter to zero

    for page in PDFPage.get_pages(document):                        #loop through the pdf page by page
        pageno = pageno + 1                                         #increment the page number
        pdf[pageno] = {}                                            #dictionary for this page
        interpreter.process_page(page)                              # receive the LTPage object for the page.
        layout = device.get_result()                                # create layout object
        tbox_no=0                                                   # index for element number
        for element in layout:
            if (type(element).__name__=='LTTextBoxHorizontal'):     #loop through text boxes
                tbox_no += 1                                        #increment text box number
                pdf[pageno][tbox_no] = {}                           #dictionary for text boxes within page
                x0 = round(element.x0,2)                            #x0 coordinate of textbox corner
                x1 = round(element.x1,2)                            #x1 coordinate of textbox corner
                y0 = round(element.y0,2)                            #y0 coordinate of textbox corner
                y1 = round(element.y1,2)                            #y1 coordinate of textbox corner
                txt = element.get_text().encode('ascii', 'ignore')  #text converted to ascii
                pdf[pageno][tbox_no]['x0'] = x0                     #create x0 coordinate entry
                pdf[pageno][tbox_no]['x1'] = x1                     #create x1 coordinate entry
                pdf[pageno][tbox_no]['y0'] = y0                     #create y0 coordinate entry
                pdf[pageno][tbox_no]['y1'] = y1                     #create y1 coordinate entry

                pdf[pageno][tbox_no]['text'] = ''.join(chr(c) for c in txt) #convert bytes to string
    return(pdf)

In [25]:
taxroll = {}              #initialize the taxroll dictionary

### Decode the taxrolls

In [26]:
def decode_taxroll(taxroll,fyear):
    avyear = fyear-2
    taxroll[fyear] = read_pdf('../tax_rolls/re_tax_rolls/EG_taxrolls/EG_RE_Tax_Roll_FY' +\
        str(fyear) + '_assessed_12_31_' + str(avyear) + '.pdf')
    print('years: ',taxroll.keys())
    return

for fy in np.arange(1995,2020+1):
    decode_taxroll(taxroll,fy)

years:  dict_keys([1995])
years:  dict_keys([1995, 1996])
years:  dict_keys([1995, 1996, 1997])
years:  dict_keys([1995, 1996, 1997, 1998])
years:  dict_keys([1995, 1996, 1997, 1998, 1999])
years:  dict_keys([1995, 1996, 1997, 1998, 1999, 2000])
years:  dict_keys([1995, 1996, 1997, 1998, 1999, 2000, 2001])
years:  dict_keys([1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002])
years:  dict_keys([1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003])
years:  dict_keys([1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004])
years:  dict_keys([1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005])
years:  dict_keys([1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006])
years:  dict_keys([1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007])
years:  dict_keys([1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008])
years:  dict_keys([1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006,

### Write taxroll dictionary to re_taxroll_decoded_pdfs_mm_dd_yyy.pkl

In [27]:
current_date = date.today()
fname = '../re_taxroll_decoded_pdfs_' + str(current_date.month) + '_' + \
    str(current_date.day) + '_' + str(current_date.year) + '.pkl'
with open(fname, 'wb') as handle:
    pickle.dump(taxroll, handle)